### Imports

In [189]:
%pylab inline

import requests
import csv
import time
import hashlib
import random
import cookielib
import sys
import re
import urllib
from  urllib2 import *
import getpass
import pandas as pd
import numpy as np

#from ghost import Ghost 
from lxml import html
from unidecode import unidecode

class Dict(dict):
    def __missing__(self, key):
            return False

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Auxiliar Methods

In [193]:
# Sleep time to avoid hard hitting Google Scholar
def sleep_and_build_opener():
    time.sleep(MIN_SLEEP + (random.random() * MAX_RANDOM_MINUTES))  
    global opener # TODO maybe 
    opener = build_opener(HTTPCookieProcessor()) 
    
def open_allowed_page():
    time.sleep(MIN_SLEEP)
    opener.open('http://scholar.google.com.br/citations?user=JMkfK0sAAAAJ&hl=pt-PT') # Berthier's page
    

def open_page(url, action):
    page = None
    try:
        sleep_and_build_opener()
        page = opener.open(url) 
        open_allowed_page()
    except URLError as e:     
        print e
        print "Error in: " + action
        return None
    except Exception as e:     
        print e
        print "Generic Exception in: " + action
        return None
    
    if page != None and page.getcode() != 200:
        print 'F**!!! Response: ' + str(page.getcode() + ' (' + action + ')')
        return None
    
    return page

def extract_element(page, xpath):
    tree = html.fromstring(page.read())
    return tree.xpath(xpath)


def extract_author_info(nrc_file, index, link, author_page):            
    tree = html.fromstring(author_page.read())

    nrc_file.ix[index, 'GS_Name'] = tree.xpath("//div[@id='gsc_prf_in']/text()")[0].encode('ascii', 'ignore')
    nrc_file.ix[index, 'GS_H5'] = tree.xpath("(//td[@class='gsc_rsb_std'])[3]/text()")[0]
    nrc_file.ix[index, 'GS_Cits'] = tree.xpath("(//td[@class='gsc_rsb_std'])[1]/text()")[0]
    nrc_file.ix[index, 'GS_H5_2010'] = tree.xpath("(//td[@class='gsc_rsb_std'])[4]/text()")[0]
    nrc_file.ix[index, 'GS_Cits_2010'] = tree.xpath("(//td[@class='gsc_rsb_std'])[2]/text()")[0]
    nrc_file.ix[index, 'GS_Link'] = link

### Not used
def authenticate():
    email = raw_input("Google username: ")
    password = getpass.getpass("Password: ")

    LOGIN_URL = 'https://accounts.google.com/ServiceLogin?service=grandcentral'
    AUTH_URL = 'https://accounts.google.com/ServiceLoginAuth?service=grandcentral'
    opener = build_opener(HTTPCookieProcessor()) 

    # Find GALX value
    login_page_contents = opener.open(LOGIN_URL).read()
    galx = re.search(r"name=\"GALX\"\s+type=\"hidden\"\s+value=\"(.+)\"", login_page_contents, re.IGNORECASE).group(1)
    # print galx 

    login_data = urllib.urlencode({
        'Email': email,
        'Passwd': password,
        'GALX': galx,
    })

    opener.open(AUTH_URL, login_data)

def parse_nrc_data(nrc_file):
    return nrc_file


### Main

In [215]:
N_AUTHORS = 1000
EXCLUDED_LASTNAMES = ["FILHO", "NETO", "JUNIOR", "JR"]
MIN_SLEEP = 2
MAX_RANDOM_MINUTES = 0
AUTHOR_URL_XPATH = "(//h3[@class='gsc_1usr_name']/a)[1]/@href"
ENCODING = 'utf-8'

opener = build_opener(HTTPCookieProcessor()) 
found_fullnames = Dict()

# NRC input file
nrc_file = pd.read_json("../nrc/cs-authors-nrc.json")
parse_nrc_data(nrc_file)

k = 0
h_index_not_found = 0
start_time = time.time()

for (index, row) in nrc_file.iterrows():
    if k % 10 == 0: print ("Progress: " + str(k) + " authors (%.1f seg)" % (time.time() - start_time))    
    k += 1 
    
    if (k > N_AUTHORS): break

    # Fullname
    full_name = unidecode(row['name'].decode(ENCODING)).upper()
    
    # Author already found
    if found_fullnames[full_name]: 
        print 'A-ha I will not search again for: ' + full_name
        continue
    
    names = full_name.split()
    lastname = names[len(names)-1]        
    if (lastname.upper() in EXCLUDED_LASTNAMES and len(names) > 2): lastname = names[len(names)-2]
    short_name = names[0] + ' ' + lastname
    # print "\n" + short_name

    # Specific Known Cases
    if full_name == 'Wagner Meira Junior':
        full_name = short_name = 'Wagner Meira'
    elif full_name == 'EDMUNDO ALBUQUERQUE DE SOUZA E SILVA':
        full_name = short_name = 'EDMUNDO SOUZA SILVA'

    # University
    university = unidecode(row['group_name'].decode(ENCODING))

    # 1st try: Fullname search
    fullname_search_url = 'http://scholar.google.com.br/citations?view_op=search_authors&mauthors=' + urllib.quote(full_name)
    search_page = open_page(fullname_search_url, 'Fullname searching')
    if search_page is None: break

    author_id = extract_element(search_page, AUTHOR_URL_XPATH)

    # 2nd try: Shortname + University search
    if not author_id: 
        shortname_search_url = 'http://scholar.google.com.br/citations?view_op=search_authors&mauthors=' + urllib.quote(short_name + ' ' + university)
        search_page = open_page(shortname_search_url, 'Shortname + University searching')
        if search_page is None: break

        author_id = extract_element(search_page, AUTHOR_URL_XPATH)

    # Giving up, jump to next author        
    if not author_id: 
        h_index_not_found += 1
    else:
        # Found author!
        author_id = author_id[0]
        author_url = "https://scholar.google.com.br" + author_id
        author_page = open_page(author_url, 'Opening author profile')
        if author_page is None: break

        found_fullnames[full_name] = True
        extract_author_info(nrc_file, index, author_id, author_page)

# End for 
nrc_file.to_csv('../h-index/nrc-h-index-bot-output.csv', encoding='utf-8')    

print("Authors Not Found in GSC: %d of %d (%.1f%%)" % (h_index_not_found, k-1, (h_index_not_found * 100.00) / (k-1)))
print("\n--- Finished after %.3f seconds. ---" % (time.time() - start_time))

del opener


Progress: 0 authors (0.0 seg)
Progress: 10 authors (125.7 seg)
Progress: 20 authors (252.8 seg)
Progress: 30 authors (425.8 seg)
Progress: 40 authors (577.3 seg)
Progress: 50 authors (1974.8 seg)
Progress: 60 authors (2100.2 seg)
Progress: 70 authors (2228.8 seg)
Progress: 80 authors (2354.3 seg)
Progress: 90 authors (2476.3 seg)
Progress: 100 authors (2600.1 seg)
Progress: 110 authors (2730.9 seg)
Progress: 120 authors (2862.3 seg)
Progress: 130 authors (2991.7 seg)
Progress: 140 authors (3104.3 seg)


UnicodeEncodeError: 'ascii' codec can't encode character u'\xf3' in position 12: ordinal not in range(128)

### NRC Json

In [214]:
nrc_file = pd.read_json("../nrc/cs-authors-nrc.json")
nrc_groupby_name = nrc_file.groupby('name')['group_name'].size()
names = nrc_groupby_name[nrc_groupby_name > 2] # without .size() again!
# print names
#nrc_file.ix[nrc_file['group_name']=='Louisiana State University and Agricultural and Mechanical College', 'name':]#['name', 'group_name', 'group_id', 'group_NRC', 'group_publication_count']]
nrc_file.ix[nrc_file['name']=='Gerald Baumgartner', :]
# nrc_file[nrc_file['group_name'] == '<<< artificial reference group >>>']

# %timeit x = 'a'


,group_NRC,group_capesMaxDMF,group_id,group_name,group_publication_count,id,name
1743,"[17, 39]",0,53e3e85631522e24717ad81b,Louisiana State University and Agricultural an...,1142,53e4e8e962c22434d8beedae,Gerald Baumgartner
6950,None,0,553cf2cd62c2242e37c7ae0f,<<< artificial reference group >>>,0,53e4e8e962c22434d8beedae,Gerald Baumgartner
